In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import tifffile as tfl

%matplotlib inline


# Data Extraction 

## train_pts is randomly scattered data over the area of interest of Yangon. The points are then being scattered and values of labels and training data are being extracted to it.

In [ ]:
train_pts = gpd.read_file('./training_data/final_training_data.shp')
train_pts


'RASTERVALU' is the value of labels from ESRI landcover

## 'composite.tif' is the composite layer stacked Landsat 9 images. You can either insert multispectral layerstacked geotif images or PCA reduced image.

In [ ]:
#Reading Raster Data 

comp = 'composite.tif'
src= rasterio.open(comp)
img = src.read()   # load our original input file bands to a numby array stack
img = img/4096.0
img = img.astype('float32')
profile = src.profile  # the copy the profile of the original GeoTIFF input file
with rasterio.io.MemoryFile() as memfile:
    with memfile.open(**profile) as dst:
        for i in range(0, src.count):
            dst.write(img[i], i+1)
    dataset = memfile.open()
 
print(img.shape)    


In [ ]:
bands = list(src.descriptions) # Uncomment this if you are using multispectral satellite image
# bands = ['PCA_1','PCA_2','PCA_3','PCA_4'] # Uncomment this if you are using PCA image 
bands

In [ ]:
# %%time
# Read points from shapefile
train_pts = gpd.read_file('./training_data/') 

train_pts = train_pts[['RASTERVALU', 'UTM_E', 'UTM_N', 'geometry']]  # These are the attributes in our point dataset
train_pts.index = range(len(train_pts))
coords = [(x,y) for x, y in zip(train_pts.UTM_E, train_pts.UTM_N)]  # Create list of point coordinates
# Sample the each band of raster dataset at each point in the coordinate list
train_pts['Raster Value'] = [x for x in dataset.sample(coords)]# all band values are saved as a list in the Raster Value column 
# Unpack the Raster Value column to separate column for each band - band names were retrieved with snappy and are now usef as column names
train_pts[bands] = pd.DataFrame(train_pts['Raster Value'].tolist(), index= train_pts.index)  
train_pts = train_pts.drop(['Raster Value'], axis=1)  # Remove Raster Value column
train_pts.to_csv(r'Finale_Training_data.csv') # save our training dataset to CSV
train_pts.head() # visualize the first rows of the dataframe 

# Creating Training Dataset

In [ ]:
x = []
x.append(train_pts[bands].values)

y = train_pts['Label'].values

In [ ]:
print(f'Shape of x array is {np.asarray(x).shape}')

In [ ]:
print(f'Shape of y array is {y.shape}')

# Create Random Forest Classifier to fit the data

In [ ]:
# Fitting Random Forest Classifier from sklearn

from sklearn.ensemble import RandomForestClassifier
model_RF = []
rf = RandomForestClassifier(n_estimators=300, oob_score=True, max_features='auto')
rf = rf.fit(x[0],y)
model_RF.append(rf)

print('Our OOB prediction of accuracy of landsat 9 is {}'.format(model_RF[0].oob_score_*100))

# Inferencing on the whole area

In [ ]:
#Reshaping geotiff image for prediction

from rasterio.plot import reshape_as_raster, reshape_as_image 
reshaped_img = reshape_as_image(img)
print('Landsat 9 Orignal image shape:',reshaped_img.shape)
reshaped_img = reshaped_img.reshape(-1,reshaped_img[2].shape[1])
print('Reshaped Image shape:',reshaped_img.shape)

In [ ]:
#Inferencing on image
pred = rf.predict(reshaped_img)
pred =pred.reshape(img.shape[1],img.shape[2])
print(pred.shape)

plt.figure(figsize=(10,10))
plt.imshow(pred,cmap='gist_earth')

## 'mask_label.tif' is an area of interest clipped image downloaded from ESRI landcover cloud. 

Link : https://www.arcgis.com/apps/instant/media/index.html?appid=fc92d38533d440078f17678ebc20e8e2

In [ ]:
#reading mask label 
mask_label ='mask_label.tif'
mask_label =tfl.imread(mask_label)
mask_label.shape

In [ ]:
from sklearn.metrics import classification_report
label = ['Classification report']
pred_flat = pred.flatten()
label_flat = mask_label.flatten()
print(label)
print(classification_report(pred_flat,label_flat,zero_division=0));


In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2,figsize=(20,10))
ax1.imshow(pred,cmap='gist_earth')
ax1.set_title('Random Forest Predictions');
ax2.imshow(mask_label,cmap='gist_earth')
ax2.set_title('Landcover Label Data from ESRI ');
